# FAISS 관리

## FAISS 저장시 생기는 파일
### 1. **.faiss 파일**
- **내용**: `.faiss` 파일은 FAISS 인덱스를 저장. 벡터 데이터에 대한 인덱스 구조와 해당 데이터를 검색하기 위한 최적화된 인덱스를 포함.
- **용도**: 인덱스를 다시 로드하여 검색 작업을 수행할 수 있다. 이 파일만 있으면 벡터 데이터에 대한 검색 작업을 다시 수행할 수 있다.

### 2. **.pkl 파일**
- **내용**: `.pkl` 파일은 일반적으로 Python에서 사용하는 피클(pickle) 파일 형식으로 주로 벡터 데이터의 메타정보 또는 인덱스와 관련된 추가 데이터가 저장. 예를 들어, 벡터에 대한 ID, 메타데이터, 인덱스 구성 설정 등이 포함된다.
- **용도**: `.pkl` 파일은 Python 객체를 직렬화하여 저장한 것으로, 인덱스를 다시 로드할 때 사용된다. 이를 통해 검색 과정에서 필요한 부가적인 정보나 설정을 복원할 수 있다.

### 요약
- **.faiss 파일**: 벡터 인덱스 데이터를 저장하는 파일로, 벡터 검색 작업에 직접 사용.
- **.pkl 파일**: Python 객체의 직렬화된 데이터를 저장하는 파일로, 인덱스와 관련된 메타정보나 설정을 포함.

이 파일들은 함께 사용되어 벡터 검색 작업을 빠르게 수행할 수 있도록 한다. 인덱스를 로드할 때 두 파일을 모두 사용하여 완전한 검색 환경을 복원할 수 있다.

In [2]:
# 한번만 수행하면 됨
# 필요한 라이브러리 설치
# !pip install chardet

# 변수 선언

In [20]:
# 경로 추가
import sys
import os
from langchain.embeddings import OllamaEmbeddings
sys.path.append(os.path.dirname(os.getcwd()))

# 변수 선언
ROOT_PATH = '/Users/passion1014/project/langchain/rag_server'
INDEX_ID = "news01" # VectorDB INDEX 명
FILE_NAME = f'{INDEX_ID}.txt' # 파일이름

# 임베딩 선언
embeddings = OllamaEmbeddings( # embeddings = OpenAIEmbeddings()
    model="nomic-embed-text",
    base_url="http://localhost:11434"  # Ollama의 기본 URL입니다. 필요에 따라 변경하세요.
)


# 데이터 저장

In [26]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import chardet
import argparse
from tools import upload_vectordb

# upload_vectordb.main(f'{ROOT_PATH}/data/{INDEX_ID}.txt', INDEX_ID)

# 파일 내용 로드 및 처리
text = upload_vectordb.load_and_detect_encoding(f'{ROOT_PATH}/data/{INDEX_ID}.txt')


# 2. 의미별로 chunk로 나누기
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=5)
chunks = text_splitter.split_text(text)

# 각 chunk를 문서 객체로 변환
docs = text_splitter.create_documents(chunks)

# FAISS 인덱스 생성 및 문서 추가
db = FAISS.from_documents(docs, embeddings)

# FAISS 인덱스 저장
        
db.save_local(f'{ROOT_PATH}/vectordb/{INDEX_ID}')
print(f"FAISS 인덱스를 '{vectordb_path}/{INDEX_ID}'에 저장했습니다.")



Created a chunk of size 202, which is longer than the specified 200
Created a chunk of size 220, which is longer than the specified 200


FAISS 인덱스를 '/Users/passion1014/project/langchain/rag_server/vectordb/news01'에 저장했습니다.


# 저장된 파일 확인

In [43]:
import pickle
import faiss

# 파일 경로
pkl_file_path = f'{ROOT_PATH}/vectordb/{INDEX_ID}/index'


# .pkl 파일 읽기
with open(f'{pkl_file_path}.pkl', 'rb') as f:
    docstore, uuid_mapping = pickle.load(f)

# 데이터를 출력하여 확인
print(f'# pkl [docstore] = {docstore}')
print(f'# pkl [uuid_mapping][{len(uuid_mapping)}] = {uuid_mapping}')


# .faiss 파일 읽기
index = faiss.read_index(f'{pkl_file_path}.faiss')

# 인덱스 정보 출력
print(f'# 인덱스 정보 출력 = {index}')

# docstore 객체의 메서드와 속성 확인
# print(dir(docstore))


# UUID로 데이터를 조회
for i in uuid_mapping:
    document = docstore.search(uuid_mapping[i])
    print(f'# 조회한 데이터 출력 = {document}')


# pkl [docstore] = <langchain_community.docstore.in_memory.InMemoryDocstore object at 0x129def110>
# pkl [uuid_mapping][19] = {0: 'a96e6d4a-d25f-4c57-88d1-d3687284d9a9', 1: '7cbf4607-74b9-483a-add2-52f9e389432f', 2: '4e43e3ee-fe1a-4f88-97ad-22a9a4197be6', 3: '9b6dd1bc-9c14-44ba-b164-6f2edf4b84b3', 4: '995216ba-4a17-40c5-ae81-4e54ce8c1099', 5: '2b84bd56-494e-4690-aa80-2a37c4145fa4', 6: '8b178345-5636-4d1d-b35a-542f525b3e05', 7: '9cbfd120-7c93-4724-9c4a-f65d0a7bffa2', 8: '5e3804ff-4180-4bd2-868e-030173144d46', 9: '3c4c9e91-2572-42cc-8508-64c2a931f867', 10: '768e3f5e-0fa9-4c0a-8b08-7a8bed0b7cb8', 11: 'ebbc4f63-8151-4ce5-832b-805c32327147', 12: '2f7d58d6-db51-4453-9dd7-04941ae209f1', 13: '4b31ba23-a31d-4720-aaee-bfa26ee57053', 14: '973c4c60-bca3-4bf3-9283-5863965aa920', 15: '720b1805-0084-43e7-a8ad-9f86805c9fc6', 16: 'cab569c3-50ee-4af3-a85a-af57981f7c51', 17: '225307f8-1d06-42f2-902d-9c2baf0d30f2', 18: 'c2a55859-2bf2-4ab1-b716-ff9072159c25'}
# 인덱스 정보 출력 = <faiss.swigfaiss.IndexFlatL2; pro

# 데이터 조회

In [21]:
from langchain_community.vectorstores import FAISS


# 파일 로드
db = FAISS.load_local(f'{ROOT_PATH}/vectordb/{INDEX_ID}', embeddings, allow_dangerous_deserialization=True)
    
retriever = db.as_retriever(search_kwargs={"k": 3})
doc = retriever.get_relevant_documents("금투세 폐지 주장한자는?")

print(doc)


[Document(page_content="증시 폭락 사태에...與는 '금투세 폐지' 발동동, 野는 무응답?"), Document(page_content='한 대표는 "증시는 심리적인 요인이 많이 반영된다. 금투세 폐지와 같은 큰 이벤트는 대개 6개월 전부터 반영되기 시작한다고 한다. 이번 폭락 때문이라도 금투세 폐지에 대한 초당적 입장을 정해야 한다"고 말했다.'), Document(page_content='김상훈 정책위의장은 당정 회의 후 브리핑에서 국내 증시 상황과 관련 "결국 우리한테는 금투세 폐지가 당면과제 아니겠느냐는 정부 측 입장이 있었고, 앞으로 주식 시장의 수요 기반이 더 강화될 수 있게 조치해 나가겠다는 정부 입장이 있었다"고 전했다.')]
